In [1]:
import numpy as np
from scipy.optimize import minimize
import math
from random import random, randint, shuffle
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression as PythonLogisticRegression
from sklearn.neighbors import KNeighborsClassifier as PythonKNN
from sklearn.svm import SVC as PythonSVM
from sklearn.tree import DecisionTreeClassifier as PythonDecisionTree
from sklearn.ensemble import RandomForestClassifier as PythonRandomForest
from sklearn import metrics
from collections import namedtuple
import heapq

In [2]:
class LogisticRegression:
    def __init__(self, epochs=100):
        self.__coefs = np.array([])
        self.__predictor = None
        self.__epochs = epochs

    def fit(self, features, targets):
        class LikehoodFunction:
            def __init__(self, x, y):
                one = np.array([1] * x.shape[0])
                self._x = np.append(one[:,np.newaxis], x, axis=1)
                self._y = y

            def __call__(self, b):
                poly = np.sum(self._x * b, axis=1)
                prob = 1 / (1 + np.exp(-poly))
                return -(np.sum(self._y * np.log(prob) + (1 - self._y) * np.log(1 - prob)))

        class LikehoodFunctionJacobian(LikehoodFunction):
            def __call__(self, b):
                poly = np.sum(self._x * b, axis=1)
                mul = np.sum(self._y - 1 / (1 + np.exp(-poly)))
                return -b * mul

        class LikehoodFunctionHessian(LikehoodFunction):
            def __call__(self, b):
                poly = np.sum(self._x * b, axis=1)
                mul = np.sum(np.exp(-poly) / np.power(1 + np.exp(-poly), 2))
                ans = b[:,np.newaxis] * b
                return ans * mul

        class Predictor:
            def __init__(self, b):
                self.__b = b

            def __call__(self, x):
                one = np.array([1] * x.shape[0])
                xa = np.append(one[:,np.newaxis], x, axis=1)
                poly = np.sum(xa * self.__b, axis=1)
                return 1 / (1 + np.exp(-poly))

        start = np.random.random(features.shape[1] + 1) - 0.5

        self.__coefs = minimize(
            LikehoodFunction(features, targets), start, method='trust-ncg', 
            jac=LikehoodFunctionJacobian(features, targets),
            hess=LikehoodFunctionHessian(features, targets),
            options={'maxiter': self.__epochs}
        ).x

        self.__predictor = Predictor(self.__coefs)
        self.__threshold = 0.5

    def predict(self, features):
        if not self.__predictor is None and self.__coefs.size == features.shape[1] + 1:
            return (self.__predictor(features) > self.__threshold) * 1

In [3]:
class KDTreeNode:
    __Neighbour = namedtuple("Neighbour", ["dist_sq", "value", "class_id"])

    def __init__(self, value=None, class_id=None, coord=0, parent=None, left=None, right=None):
        self.value = value
        self.class_id = class_id
        self.coord = coord
        self.left = left
        self.right = right
        self.parent = parent
        
    def find_closest_k(self, target, k):
        ans = []

        if target[self.coord] >= self.value[self.coord]:
            if not self.right is None:
                ans = self.right.find_closest_k(target, k)
            neighbour = self.left
        else:
            if not self.left is None:
                ans = self.left.find_closest_k(target, k)
            neighbour = self.right

        curr_dist = np.sum(np.power(target - self.value, 2))

        if len(ans) == 0 or curr_dist < heapq.nlargest(1, ans)[0].dist_sq:
            pushed = self.__Neighbour(curr_dist, self.value, self.class_id)
            if len(ans) < k:
                heapq.heappush(ans, pushed)
            else:
                heapq.heappushpop(ans, pushed)

        dif = np.power(target[self.coord] - self.value[self.coord], 2)

        if not neighbour is None and dif <= heapq.nlargest(1, ans)[0].dist_sq:
            ans += neighbour.find_closest_k(target, k)
            heapq.heapify(ans)
            if len(ans) > k:
                ans = heapq.nsmallest(k, ans)
                heapq.heapify(ans)

        return ans

class KDTree:
    def __init__(self, numbers=None, classes=None):
        if not numbers is None and not classes is None:
            self.__root = self.__construct(numbers, classes, 0, len(numbers) - 1, None)
        else:
            self.__root = None

    def __construct(self, numbers, classes, left, right, parent):
        if left > right:
            return None

        if left == right:
            return KDTreeNode(
                value=numbers[left],
                class_id=classes[left]
            )

        max_diffed_coords_ind = np.argmax([len(np.unique(i)) for i in numbers[left:right + 1].transpose()])

        median = (left + right) // 2
        while median > left and numbers[median, max_diffed_coords_ind] == numbers[median - 1, max_diffed_coords_ind]:
            median -= 1

        ans = KDTreeNode(
            coord=max_diffed_coords_ind,
            value=numbers[median],
            class_id=classes[median],
            parent=parent
        )

        ans.left = self.__construct(numbers, classes, left, median - 1, ans)
        ans.right = self.__construct(numbers, classes, median + 1, right, ans)

        return ans

    def find_closest_k(self, target, k):
        return self.__root.find_closest_k(target, k)

    def empty(self):
        return self.__root is None

class KNN:
    def __init__(self, k=50):
        self.__tree = KDTree()
        self.__k = k

    def fit(self, features, targets):
        self.__tree = KDTree(features, targets)

    def predict(self, points):
        ans = np.array([])
        for point in points:
            res = self.__tree.find_closest_k(point, self.__k)

            weights = {}
            for neighbour in res:
                if not neighbour.class_id in weights:
                    weights[neighbour.class_id] = 0
                weights[neighbour.class_id] += 1 / neighbour.dist_sq

            res_class = None
            for class_id in weights:
                if res_class is None or weights[res_class] < weights[class_id]:
                    res_class = class_id

            ans = np.append(ans, res_class)

        return ans

In [4]:
class SVM():
    def __init__(self, epochs=5, a=0.5):
        self.__epochs = epochs
        self.__a = a
        self.__predictor = None

    def fit(self, features, targets):
        class Q:
            def __init__(self, x, y, a):
                one = np.array([1] * x.shape[0])
                self._x = np.append(one[:,np.newaxis], x, axis=1)
                self._y = y
                self._a = a
                self._i = 0

            def _iterate(self):
                self._i += 1
                if self._i == len(self._x):
                    self._i = 0

            def __call__(self, w):
                ans = max(0., 1. - self._y[self._i] * np.sum(w * self._x[self._i])) + self._a * np.sum(w * w) / 2
                self._iterate()
                return ans

        class QJac(Q):
            def __call__(self, w):
                if self._y[self._i] * np.sum(w * self._x[self._i]) < 1:
                    ans = self._a * w - self._y[self._i] * self._x[self._i]
                else:
                    ans = self._a * w
                self._iterate()
                return ans

        class Predictor:
            def __init__(self, w):
                self.__w = w

            def __call__(self, x):
                one = np.array([1] * x.shape[0])
                xa = np.append(one[:,np.newaxis], x, axis=1)
                return np.sum(self.__w * xa, axis=1)

        start = np.random.random(features.shape[1] + 1) - 0.5

        q_func = Q(features, targets, self.__a)
        q_jac = QJac(features, targets, self.__a)

        for i in range(self.__epochs):
            for j in range(features.shape[0]):
                self.__coefs = minimize(
                    q_func, start, method='CG', 
                    jac=q_jac,
                    options={'maxiter': 1}
                ).x

        self.__predictor = Predictor(self.__coefs)

    def predict(self, features):
        if not self.__predictor is None and self.__coefs.size == features.shape[1] + 1:
            return (self.__predictor(features) > 0) * 1

In [5]:
class DecisionTreeLeaf:
    def __init__(self, decision):
        self.__decision = decision

    def predict(self, point):
        return self.__decision

class DecisionTreeNode:
    def __init__(self, feature, separator, left, right):
        self.__feature = feature
        self.__separator = separator
        self.__left = left
        self.__right = right

    def predict(self, point):
        if point[self.__feature] <= self.__separator:
            return self.__left.predict(point)
        else:
            return self.__right.predict(point)

class DecisionTree:
    def __init__(self, depth=None):
        self.__root = None
        self.__depth = depth

    def fit(self, features, targets):
        self.__root = self.__construct(features, targets)

    def predict(self, points):
        if self.__root is not None:
            ans = []
            for point in points:
                ans.append(self.__root.predict(point))
            return np.array(ans)

    def __construct(self, features, targets, depth=0):
        unique_targets = np.unique(targets)
        if unique_targets.size == 1:
            return DecisionTreeLeaf(unique_targets[0])

        if not self.__depth is None and self.__depth == depth:
            return DecisionTreeLeaf(self.__get_freq_target(targets))

        max_gain = None
        for i in range(features.shape[1]):
            curr_gain, curr_sep = self.__information_gain(features[:,i], targets)
            if max_gain is None or curr_gain > max_gain:
                max_ind = i
                max_sep = curr_sep
                max_gain = curr_gain

        left = ([], [])
        right = ([], [])
        for i in range(features.shape[0]):
            if features[i, max_ind] <= max_sep:
                left[0].append(features[i])
                left[1].append(targets[i])
            else:
                right[0].append(features[i])
                right[1].append(targets[i])

        if len(left[0]) == 0 or len(right[0]) == 0:
            return DecisionTreeLeaf(self.__get_freq_target(targets))

        return DecisionTreeNode(
            feature=max_ind,
            separator=max_sep,
            left=self.__construct(np.array(left[0]), np.array(left[1]), depth + 1),
            right=self.__construct(np.array(right[0]), np.array(right[1]), depth + 1)
        
        )

    def __information_gain(self, features, targets):
        targets_counts = {}
        for target in targets:
            if target not in targets_counts:
                targets_counts[target] = 0
            targets_counts[target] += 1

        sorted_targets_inds = np.argsort(features)
        values = np.sort(features)

        optimal_separator = None
        curr_targets_counts = {}
        for target in targets_counts:
            curr_targets_counts[target] = 0

        for i in range(len(targets) - 1):
            curr_targets_counts[targets[sorted_targets_inds[i]]] += 1

            curr_information_ammount = 0
            for target in targets_counts:
                p1 = curr_targets_counts[target] / (i + 1)
                p2 = (targets_counts[target] - curr_targets_counts[target]) / (len(targets) - (i + 1))

                if p1 > 0:
                    curr_information_ammount -= p1 * math.log(p1, 2)
                if p2 > 0:
                    curr_information_ammount -= p2 * math.log(p2, 2)

            if optimal_separator is None or curr_information_ammount < optimal_information_ammount:
                optimal_separator = values[i]
                optimal_information_ammount = curr_information_ammount

        base_information_ammount = 0
        for target_count in targets_counts.values():
            p = target_count / len(features)
            base_information_ammount -= p * math.log(p, 2)

        return (base_information_ammount - optimal_information_ammount, optimal_separator)

    def __get_freq_target(self, targets):
        targets_count = {}

        for target in targets:
            if not target in targets_count:
                targets_count[target] = 0
            targets_count[target] += 1

        best_target = None
        for curr_target in targets_count:
            if best_target is None or targets_count[best_target] < targets_count[curr_target]:
                best_target = curr_target

        return best_target

class RandomForest:
    def __init__(self, trees_count=None, depth=None, effective_factors=None):
        self.__trees_count = trees_count
        self.__depth = depth
        self.__effective_factors = effective_factors
        self.__trees = None

    def fit(self, features, targets):
        if self.__effective_factors is None:
            self.__effective_factors = math.floor(math.sqrt(features.shape[1]))
        if self.__effective_factors > features.shape[1]:
            self.__effective_factors = features.shape[1]

        if self.__trees_count is None:
            self.__trees_count = math.ceil(math.sqrt(features.shape[1]))

        self.__trees = []

        for i in range(self.__trees_count):
            points_inds = [randint(0, features.shape[0] - 1) for i in range(features.shape[0])]
            tmp = [i for i in range(features.shape[1])]
            shuffle(tmp)
            targets_inds = sorted(tmp[:self.__effective_factors])

            curr_points = []
            curr_targets = []
            for ind in points_inds:
                curr_points.append(features[ind])
                curr_targets.append(targets[ind])

            curr_points = np.array(curr_points)
            curr_targets = np.array(curr_targets)

            trunc_factors = curr_points[:, targets_inds[0], np.newaxis]
            for ind in targets_inds[1:]:
                np.append(trunc_factors, curr_points[:, ind, np.newaxis], axis=1)

            curr_tree = DecisionTree(self.__depth)
            curr_tree.fit(trunc_factors, curr_targets)
            self.__trees.append(curr_tree)

    def predict(self, points):
        ans = []
        for point in points:
            votes = {}
            for tree in self.__trees:
                pred = tree.predict([point])[0]
                if pred not in votes:
                    votes[pred] = 0
                votes[pred] += 1

            predicted_class = None
            for class_id, votes_count in votes.items():
                if predicted_class is None or votes[predicted_class] < votes_count:
                    predicted_class = class_id

            ans.append(predicted_class)
        return np.array(ans)

In [6]:
def load_csv(path):
    csv_path = os.path.join(path)
    return pd.read_csv(csv_path)

In [7]:
data = load_csv("E:\Labs\AI\Datasets\weather_ready_full.csv")

In [8]:
probs = np.random.rand(len(data))
sampling_rate = 0.8
training_mask = probs < sampling_rate
test_mask = probs >= sampling_rate

learn_data = data[training_mask]
learn_features = learn_data.drop("RainTomorrow", axis='columns').to_numpy()
learn_targets = learn_data["RainTomorrow"].to_numpy()

test_data = data[test_mask]
test_features = test_data.drop("RainTomorrow", axis='columns').to_numpy()
test_targets = test_data["RainTomorrow"].to_numpy()

In [9]:
def learn_and_calc_metrics(model, learn_features, learn_targets, test_features, test_targets, name):
    model.fit(learn_features, learn_targets)
    res = model.predict(test_features)
    print("{} metrics:".format(name))
    print("accuracy:", metrics.accuracy_score(test_targets, res))
    print("precision:", metrics.precision_score(test_targets, res))
    print("f1 score:", metrics.f1_score(test_targets, res))
    print("recall:", metrics.recall_score(test_targets, res))
    print()

In [10]:
learn_and_calc_metrics(LogisticRegression(), learn_features, learn_targets, test_features, test_targets, "My logistic regression")

E:\Labs\conda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
E:\Labs\conda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply


My logistic regression metrics:
accuracy: 0.773655763021199
precision: 0.773655763021199
f1 score: 0.8723854754130801
recall: 1.0



In [11]:
learn_and_calc_metrics(PythonLogisticRegression(), learn_features, learn_targets, test_features, test_targets, "Python logistic regression")

Python logistic regression metrics:
accuracy: 0.773655763021199
precision: 0.773655763021199
f1 score: 0.8723854754130801
recall: 1.0



In [12]:
knn_sampling_rate = 0.01

knn_learn_probs = np.random.rand(len(learn_features))
taken_mask = knn_learn_probs < knn_sampling_rate
knn_learn_features = learn_features[taken_mask]
knn_learn_targets = learn_targets[taken_mask]

knn_test_probs = np.random.rand(len(test_features))
taken_mask = knn_test_probs < knn_sampling_rate
knn_test_features = test_features[taken_mask]
knn_test_targets = test_targets[taken_mask]

learn_and_calc_metrics(KNN(), knn_learn_features, knn_learn_targets, knn_test_features, knn_test_targets, "My KNN")

My KNN metrics:
accuracy: 0.717391304347826
precision: 0.808695652173913
f1 score: 0.8266666666666667
recall: 0.8454545454545455



In [13]:
learn_and_calc_metrics(PythonKNN(), learn_features, learn_targets, test_features, test_targets, "Python KNN")

Python KNN metrics:
accuracy: 0.7956619401937386
precision: 0.8395645802805107
f1 score: 0.8732363699703884
recall: 0.9097219071814181



In [22]:
learn_and_calc_metrics(SVM(1), learn_features, learn_targets, test_features, test_targets, "My SVM")

My SVM metrics:
accuracy: 0.773655763021199
precision: 0.773655763021199
f1 score: 0.8723854754130801
recall: 1.0



In [21]:
learn_and_calc_metrics(PythonSVM(max_iter=1000), learn_features, learn_targets, test_features, test_targets, "Python SVM")

E:\Labs\conda\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Python SVM metrics:
accuracy: 0.6251579390706163
precision: 0.7757875831270327
f1 score: 0.7495544508019886
recall: 0.7250374268475253



In [25]:
learn_and_calc_metrics(DecisionTree(), learn_features, learn_targets, test_features, test_targets, "My decision tree")

My decision tree metrics:
accuracy: 1.0
precision: 1.0
f1 score: 1.0
recall: 1.0



In [17]:
learn_and_calc_metrics(PythonDecisionTree(), learn_features, learn_targets, test_features, test_targets, "Python decision tree")

Python decision tree metrics:
accuracy: 1.0
precision: 1.0
f1 score: 1.0
recall: 1.0



In [18]:
learn_and_calc_metrics(RandomForest(depth=6), learn_features, learn_targets, test_features, test_targets, "My random forest")

My random forest metrics:
accuracy: 0.773655763021199
precision: 0.773655763021199
f1 score: 0.8723854754130801
recall: 1.0



In [19]:
learn_and_calc_metrics(PythonRandomForest(), learn_features, learn_targets, test_features, test_targets, "Python random forest")

Python random forest metrics:
accuracy: 0.9999649024287519
precision: 0.9999546361821811
f1 score: 0.9999773175766099
recall: 1.0

